In [14]:
import json
import warnings
import pandas as pd
from pytorch_forecasting import Baseline, DeepAR, TimeSeriesDataSet
from pytorch_forecasting.data import NaNLabelEncoder
from forecast import *

warnings.filterwarnings("ignore")


class deepAR_model:
    def __init__(self, pl_ckpt, context_length, predictor_length, building):
        self.model = DeepAR.load_from_checkpoint(pl_ckpt)
        self.building_series = building
        self.context_length = context_length
        self.predictor_length = predictor_length

    def predict(self, history_data):
        data = pd.read_csv(history_data)
        data = data.drop(['Wind', 'Precip.', 'Wind Gust'], axis=1)
        data = data.dropna()
        
        cutoff = data["time_idx"].max() - self.predictor_length

        print(f'The size of data is {len(data[lambda x: x["time_idx"] <= cutoff])}')

        print(f'The cutoff is {cutoff}')    # 335
        print(f'The max of time_idx is {data["time_idx"].max()}')   # 503
        print(f'The context length is {self.context_length}')   # 336
        print(f'The predictor length is {self.predictor_length}')   # 168
        return data
        history = TimeSeriesDataSet(
            data[lambda x: x.index <= cutoff],
            time_idx="time_idx",
            target="val",
            categorical_encoders={"Building": NaNLabelEncoder().fit(data.Building)},
            group_ids=["Building"],
            static_categoricals=["Building"],
            time_varying_known_reals=["Temperature", "Humidity", "Pressure"],
            allow_missing_timesteps=True,
            time_varying_unknown_reals=["val"],
            max_encoder_length=self.context_length,
            max_prediction_length=self.predictor_length,
            train = False
        )

        test = TimeSeriesDataSet.from_dataset(history, data, min_prediction_idx=cutoff + 1)
        batch_size = 128
        test_dataloader = test.to_dataloader(train=False, batch_size=batch_size, num_workers=0,
                                             batch_sampler='synchronized')

        predictions = self.model.predict(test_dataloader)
        pred_dict = {}
        for idx in range(len(self.building_series)):
            pred_dict[self.building_series[idx]] = predictions[idx].tolist()

        with open("prediction.json", "w") as f:
            json.dump(pred_dict, f)

csv_path = "../data/weather/future/predict_data.csv"
model_path = './my_model/epoch=23-step=1200.ckpt'
pred_day = str(20221124)
num_day_context = 7
num_day_pred = 1
building = ['1A', '1B', '1C', '1D', '1E', '2A', '2B', '2C', '2D', '2E']
model = deepAR_model(model_path, 24 * num_day_context, 24 * num_day_pred, building)

data = model.predict(csv_path)
data

Lightning automatically upgraded your loaded checkpoint from v1.7.0 to v2.0.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file my_model/epoch=23-step=1200.ckpt`


The size of data is 1670
The cutoff is 167
The max of time_idx is 191
The context length is 336
The predictor length is 24


,timestamp,Temperature,Dew Point,Humidity,Wind Speed,Pressure,Condition,val,Building,time_idx
0,2022-11-17 00:30:00,59.0,57.0,94.0,2.0,30.04,Light Rain,14.060059,1A,0
1,2022-11-17 01:30:00,61.0,59.0,94.0,2.0,30.04,Fog,12.856445,1A,1
2,2022-11-17 02:30:00,61.0,59.0,94.0,4.0,30.04,Fog,13.673096,1A,2
3,2022-11-17 03:30:00,61.0,59.0,94.0,7.0,30.04,Fog,13.463135,1A,3
4,2022-11-17 04:30:00,63.0,59.0,88.0,4.0,30.01,Fog,13.476563,1A,4
...,...,...,...,...,...,...,...,...,...,...
1915,2022-11-24 19:30:00,63.0,54.0,72.0,4.0,30.10,Partly Cloudy,0.000000,2E,187
1916,2022-11-24 20:30:00,63.0,55.0,77.0,2.0,30.10,Mostly Cloudy,0.000000,2E,188
1917,2022-11-24 21:30:00,63.0,54.0,72.0,0.0,30.10,Fair,0.000000,2E,189
1918,2022-11-24 22:30:00,63.0,55.0,77.0,4.0,30.10,Cloudy,0.000000,2E,190


Lightning automatically upgraded your loaded checkpoint from v1.7.0 to v2.0.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file my_model/epoch=23-step=1200.ckpt`


read csv file from ../data/weather/future/predict_data.csv
The size of data is 3350
The cutoff is 335
The max of time_idx is 503
The context length is 336
The predictor length is 168


AssertionError: filters should not remove entries all entries - check encoder/decoder lengths and lags